### Simple GenAI App using langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGCHAIN_ENDPOINT")
os.environ["LANGCHAIN_PROJECT"] = "webpage-qna-using-langchain"

#### Step 1:- Load the documents

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.ibm.com/think/topics/langgraph")
docs = loader.load()
docs[0].page_content

USER_AGENT environment variable not set, consider setting it to identify your requests.


"\n\n\n\n\n\n\n\n\nWhat is LangGraph? | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                   \n\n\n\n  \n    What is LangGraph?\n\n\n\n\n\n\n    \n\n\n                               \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAI Agents\n\n\n\nWelcome\n\n\n\n\n\nCaret right\n\nIntroduction\n\n\n\n\nOverview\n\n\n\n\nAI agents vs AI assistants\n\n\n\n\nAgentic AI\n\n\n\n\nAgentic AI vs generative AI\n\n\n\n\nTypes of AI agents\n\n\n\n\n\n\n\nCaret right\n\nComponents\n\n\n\n\nOverview\n\n\n\n\nPerception\n\n\n\n\nReasoning\n\n\n\n\nMemory\n\n\n\n\nPlanning\n\n\n\n\n\nCaret right\n\nTool calling\n\n\n\n\nWhat is tool calling?\n\n\n\n\nTutorial: Ollama tool calling\n\n\n\n\n\n\nCommunication\n\n\n\n\nLearning\n\n\n\n\nAgentic workflows\n\n\n\n\n\n\n\nCaret right\n\nArchitecture\n\n\n\n\nOverview\n\n\n\n\nReAct\n\n\n\n\n\n

#### Step 2:- Split it into chunks

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
splits

[Document(metadata={'source': 'https://www.ibm.com/think/topics/langgraph', 'title': 'What is LangGraph? | IBM', 'description': 'Stay ahead of the curve with AI-powered solutions that enhance productivity and streamline workflows using WatsonX and LangGraph.', 'language': 'en'}, page_content='What is LangGraph? | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                   \n\n\n\n  \n    What is LangGraph?\n\n\n\n\n\n\n    \n\n\n                               \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAI Agents\n\n\n\nWelcome\n\n\n\n\n\nCaret right\n\nIntroduction\n\n\n\n\nOverview\n\n\n\n\nAI agents vs AI assistants\n\n\n\n\nAgentic AI\n\n\n\n\nAgentic AI vs generative AI\n\n\n\n\nTypes of AI agents\n\n\n\n\n\n\n\nCaret right\n\nComponents\n\n\n\n\nOverview\n\n\n\n\nPerception\n\n\n\n\nReasoning\n\n\n\n\nMemory\n\n\n\n\nPl

#### Step 3:- Use Embedding techniques to convert the documnets into vectors

In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

#### Step 4:- Store the documents and embeding into VectorDB. Here using FAISS

In [6]:
from langchain_community.vectorstores import FAISS

vectorStoredb = FAISS.from_documents(splits,embeddings)

In [7]:
vectorStoredb

#### Step 5:- Qurey from a vectordb (Similarity search)

In [ ]:
query = "Edges: Edges are a function within Python that determines which node to execute next based on the current state. Edges can be conditional branches or fixed transitions."
result = vectorStoredb.similarity_search_with_relevance_scores(query)
result[0]

(Document(id='e3be118b-a2b5-4a6a-a33a-ceab41af9489', metadata={'source': 'https://www.ibm.com/think/topics/langgraph', 'title': 'What is LangGraph? | IBM', 'description': 'Stay ahead of the curve with AI-powered solutions that enhance productivity and streamline workflows using WatsonX and LangGraph.', 'language': 'en'}, page_content='Nodes: In LangGraph, nodes represent individual components or agents within an AI workflow. Nodes can be thought of as “actors” that interact with each other in a specific way. For example,\xa0to add nodes for tool calling, one can use the ToolNode. Another example, the next node, refers to the node that will be executed following the current one.\nEdges: Edges are a function within Python that determines which node to execute next based on the current state. Edges can be conditional branches or fixed transitions.'),
 np.float32(0.84245634))

#### Step 6:- Create LLM

In [12]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [13]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
<context>

"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n<context>\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000228A78F0EC0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000228A78F3A10>, root_client=<openai.OpenAI object at 0x00000228A792B750>, root_async_client=<openai.AsyncOpenAI object at 0x00000228A792BD90>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={

In [15]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Edges: Edges are a function within Python that determines which node to execute next based on the current state.",
    "context": [Document(page_content="Edges: Edges are a function within Python that determines which node to execute next based on the current state. Edges can be conditional branches or fixed transitions.")]
})

'Edges in Python, as described in the context, determine which node to execute next based on the current state. They can either be conditional branches, where the transition to the next node depends on certain conditions being met, or fixed transitions, where the next node is predetermined.'

#### Step 7:- Use retriever

In [17]:
retriever = vectorStoredb.as_retriever()
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [18]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022844E14590>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n<context>\n\n'), additional_kwargs={})])
            | Cha

#### Step 8:- Get the response from the llm

In [19]:
response = retrieval_chain.invoke(
    {
        "input":"Edges: Edges are a function within Python that determines which node to execute next based on the current state."
    }
)
response['answer']

"How does LangGraph facilitate the management of AI workflows and decision-making processes?\n\nLangGraph facilitates the management of AI workflows and decision-making processes by using a graph-based architecture, which allows for transparent tracking and management of AI states and decisions through nodes and edges. Nodes represent components or agents, and edges determine the flow of execution. LangGraph's stateful graph concept retains information about previous steps, enabling continuous and contextual processing. It supports cyclical graphs for complex workflows with cyclic dependencies. Enhanced decision-making is achieved through modeling complex relationships between nodes, offering increased flexibility and scalability in AI workflows. By enabling observability and state management, LangGraph helps in understanding system processes, useful for debugging and insights."

In [20]:
response

{'input': 'Edges: Edges are a function within Python that determines which node to execute next based on the current state.',
 'context': [Document(id='e3be118b-a2b5-4a6a-a33a-ceab41af9489', metadata={'source': 'https://www.ibm.com/think/topics/langgraph', 'title': 'What is LangGraph? | IBM', 'description': 'Stay ahead of the curve with AI-powered solutions that enhance productivity and streamline workflows using WatsonX and LangGraph.', 'language': 'en'}, page_content='Nodes: In LangGraph, nodes represent individual components or agents within an AI workflow. Nodes can be thought of as “actors” that interact with each other in a specific way. For example,\xa0to add nodes for tool calling, one can use the ToolNode. Another example, the next node, refers to the node that will be executed following the current one.\nEdges: Edges are a function within Python that determines which node to execute next based on the current state. Edges can be conditional branches or fixed transitions.'),
  

In [21]:
response['context']

[Document(id='e3be118b-a2b5-4a6a-a33a-ceab41af9489', metadata={'source': 'https://www.ibm.com/think/topics/langgraph', 'title': 'What is LangGraph? | IBM', 'description': 'Stay ahead of the curve with AI-powered solutions that enhance productivity and streamline workflows using WatsonX and LangGraph.', 'language': 'en'}, page_content='Nodes: In LangGraph, nodes represent individual components or agents within an AI workflow. Nodes can be thought of as “actors” that interact with each other in a specific way. For example,\xa0to add nodes for tool calling, one can use the ToolNode. Another example, the next node, refers to the node that will be executed following the current one.\nEdges: Edges are a function within Python that determines which node to execute next based on the current state. Edges can be conditional branches or fixed transitions.'),
 Document(id='9018cb7a-14d2-4e39-8627-2a2e4f9328b4', metadata={'source': 'https://www.ibm.com/think/topics/langgraph', 'title': 'What is Lan